Make video from photos in directory, based on photos, determined by time of recording

Print date and time on top of video

Indicate frames where the bee was rewarded


Callin Switzer

11 Dec 2016




In [1]:
import cv2
import numpy as np
import matplotlib
matplotlib.use("TkAgg") # have to use this for tkinter to  work below
from matplotlib import pyplot as plt
%matplotlib tk

# scikit image
import skimage
from skimage import io

import os
import pandas as pd
import time

from datetime import datetime

/Users/callinswitzer/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
%qtconsole

In [3]:
import Tkinter as tk
import tkFileDialog

def file_choose():
    '''
    Opens tkfileDialog and returns full path name.
    It's the equivalent of file.choose in R.
    Make sure the following modeules are loaded
    import Tkinter as tk
    import tkFileDialog
    Also, won't work if %matplotlib qt4 has been used
    '''
    root = tk.Tk()
    root.update()
    file1 = tkFileDialog.askopenfilename(parent=root)
    root.destroy() # closes the window
    return file1

In [4]:
# adjust gamma
def adjust_gamma(image, gamma=1.0):
    # build a lookup table mapping the pixel values [0, 255] to
    # their adjusted gamma values
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")
 
    # apply gamma correction using the lookup table
    return cv2.LUT(image, table)

In [5]:
os.chdir(("/Users/callinswitzer/Dropbox/ExperSummer2016/"
            "BeeSonicationLearningWithAvery/"
             "BeeFrequencyLearning/"))

In [7]:
beeTrialList =  [ f for f in os.listdir(".") if not f.startswith(r".") ]
beeTrialList[0:5]

['BeeBlue2_28Nov2016_Hive4_high',
 'BeeBlue3_28Nov2016_Hive4_high',
 'BeeBlue4_28Nov2016_Hive4_high',
 'BeeBlue5_28Nov2016_Hive4_high',
 'BeeBlue6_28Nov2016_Hive4_high']

In [8]:
trialNum = 51
CurrentTrialDir = os.path.join( os.getcwd() , beeTrialList[trialNum])
trialFolder = [ f for f in os.listdir(CurrentTrialDir) if not f.endswith(r".txt") and not f.startswith(r".") ]

trialMeta = [ f for f in os.listdir(CurrentTrialDir) if  f.endswith(r".txt") and not f.startswith(r".") ]
tm1 = pd.read_csv(os.path.join(CurrentTrialDir, trialMeta[0] ), header = None)

# replace leading spaces
tm1.ix[:,2] = [cc.replace(" ", "") for cc in tm1.ix[:,2]]

trialFolderDir = os.path.join(CurrentTrialDir, trialFolder[0])
photoList = [p for p in os.listdir(trialFolderDir) if p.endswith(r".pgm")]

# remove background photo from photolist
photoList.remove('background.pgm')

In [9]:
# sort list by date created
creDate = [os.stat(os.path.join(trialFolderDir, c)).st_mtime for c in  photoList]
creD_time = [datetime.fromtimestamp(aa).strftime('%Y-%m-%d %H:%M:%S') for aa in creDate]

sortedPhotos = np.array(photoList)[np.argsort(np.array(creDate))]

photoTimes = [c[:24] for c in sortedPhotos]



In [10]:
# rewards
rewArr = np.array(tm1.ix[:,2][tm1.ix[:,4] == " T"])

rewd1 = [photoTimes[kk] in rewArr for kk in range(len(photoTimes))]

In [11]:
# label rewards -- 
rr = np.cumsum(rewd1)

In [12]:
# make file with metadata from video
dd = pd.DataFrame({'photoName':sortedPhotos, 'photoTime':photoTimes,
                   'vidFrame':range(1, len(sortedPhotos) + 1), 
                  'dateModified':np.array(creD_time)[np.argsort(np.array(creDate))], 
                  'rewardNum':rr, 
                  'rewardTF': rewd1})
dd




,dateModified,photoName,photoTime,rewardNum,rewardTF,vidFrame
0,2016-12-12 09:13:28,2016_12_12__09_13_28_510_bkg.pgm,2016_12_12__09_13_28_510,0,False,1
1,2016-12-12 09:13:29,2016_12_12__09_13_29_209_bkg.pgm,2016_12_12__09_13_29_209,0,False,2
2,2016-12-12 09:13:29,2016_12_12__09_13_29_909_bkg.pgm,2016_12_12__09_13_29_909,0,False,3
3,2016-12-12 09:13:30,2016_12_12__09_13_30_623_bkg.pgm,2016_12_12__09_13_30_623,0,False,4
4,2016-12-12 09:13:31,2016_12_12__09_13_31_323_bkg.pgm,2016_12_12__09_13_31_323,0,False,5
5,2016-12-12 09:13:32,2016_12_12__09_13_32_066_bkg.pgm,2016_12_12__09_13_32_066,0,False,6
6,2016-12-12 09:13:32,2016_12_12__09_13_32_766_bkg.pgm,2016_12_12__09_13_32_766,0,False,7
7,2016-12-12 09:13:33,2016_12_12__09_13_33_556_bkg.pgm,2016_12_12__09_13_33_556,0,False,8
8,2016-12-12 09:13:34,2016_12_12__09_13_34_250_bkg.pgm,2016_12_12__09_13_34_250,0,False,9
9,2016-12-12 09:13:34,2016_12_12__09_13_34_949_bkg.pgm,2016_12_12__09_13_34_949,0,False,10


In [20]:
photoPaths = [os.path.join(trialFolderDir, c) for c in  sortedPhotos]

# sorted photo paths

kk = 0

# change directory
newD = '/Users/callinswitzer/Desktop/testStack/'
if not os.path.isdir(newD):
    os.mkdir(newD)


In [22]:
for kk in range(len(sortedPhotos)):
    img = cv2.imread(photoPaths[kk], 0)


    s1 =  sortedPhotos[kk].split(".")[0]
    try:
        s2 = s1.split("__")
        s2[0] = s2[0].replace('_', '-')
        s2[1] = s2[1][:8].replace('_', ':') + '.' + s2[1][9:12]
        s3 = s2[0] + " " + s2[1]
    except:
        s3 = "No Timestamp"

    plt.clf()
    io.imshow(img)
    plt.axis([0, img.shape[1], img.shape[0], 0])
    plt.axis('off')

    plt.gca().set_position([0,0,1,1])
    plt.gca().set_aspect('equal')
    plt.text(850, 60, s3)
    #if s2[1]
    if dd['rewardTF'][kk]:
        plt.text(850, 90, "REWARD " + str(rr[kk]))


    # save figure
    plt.savefig(newD + str(kk).zfill(4) + '.png')
    print str(kk) + ' of ' + str(len(sortedPhotos))





0 of 512
1 of 512
2 of 512
3 of 512
4 of 512
5 of 512
6 of 512
7 of 512
8 of 512
9 of 512
10 of 512
11 of 512
12 of 512
13 of 512
14 of 512
15 of 512
16 of 512
17 of 512
18 of 512
19 of 512
20 of 512
21 of 512
22 of 512
23 of 512
24 of 512
25 of 512
26 of 512
27 of 512
28 of 512
29 of 512
30 of 512
31 of 512
32 of 512
33 of 512
34 of 512
35 of 512
36 of 512
37 of 512
38 of 512
39 of 512
40 of 512
41 of 512
42 of 512
43 of 512
44 of 512
45 of 512
46 of 512
47 of 512
48 of 512
49 of 512
50 of 512
51 of 512
52 of 512
53 of 512
54 of 512
55 of 512
56 of 512
57 of 512
58 of 512
59 of 512
60 of 512
61 of 512
62 of 512
63 of 512
64 of 512
65 of 512
66 of 512
67 of 512
68 of 512
69 of 512
70 of 512
71 of 512
72 of 512
73 of 512
74 of 512
75 of 512
76 of 512
77 of 512
78 of 512
79 of 512
80 of 512
81 of 512
82 of 512
83 of 512
84 of 512
85 of 512
86 of 512
87 of 512
88 of 512
89 of 512
90 of 512
91 of 512
92 of 512
93 of 512
94 of 512
95 of 512
96 of 512
97 of 512
98 of 512
99 of 512
100 of 512

In [23]:
oldDir = os.getcwd()

In [24]:
os.chdir(newD)
# use ffmpeg to convert directory to video
stta = time.time()
!ffmpeg -start_number 0 -r 24 -i %04d.png -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" -c:v libx264 -pix_fmt yuv420p -y test51.mp4
print time.time() - stta # ~ 1.4 seconds

ffmpeg version 3.1.4 Copyright (c) 2000-2016 the FFmpeg developers
  built with Apple LLVM version 8.0.0 (clang-800.0.38)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/3.1.4 --enable-shared --enable-pthreads --enable-gpl --enable-version3 --enable-hardcoded-tables --enable-avresample --cc=clang --host-cflags= --host-ldflags= --enable-opencl --enable-libx264 --enable-libmp3lame --enable-libxvid --disable-lzma --enable-vda
  libavutil      55. 28.100 / 55. 28.100
  libavcodec     57. 48.101 / 57. 48.101
  libavformat    57. 41.100 / 57. 41.100
  libavdevice    57.  0.101 / 57.  0.101
  libavfilter     6. 47.100 /  6. 47.100
  libavresample   3.  0.  0 /  3.  0.  0
  libswscale      4.  1.100 /  4.  1.100
  libswresample   2.  1.100 /  2.  1.100
  libpostproc    54.  0.100 / 54.  0.100
Input #0, image2, from '%04d.png':
  Duration: 00:00:20.48, start: 0.000000, bitrate: N/A
    Stream #0:0: Video: png, rgba(pc), 800x600 [SAR 3937:3937 DAR 4:3], 25 fps, 25 tbr, 25 tbn, 25 tbc
[libx264

In [25]:
# delete all png files in directory
filelist = [ f for f in os.listdir(newD) if f.endswith(".png") ]

for f in filelist:
    os.remove(f)

In [ ]:
creD_time = [datetime.fromtimestamp(aa).strftime('%Y-%m-%d %H:%M:%S') for aa in creDate]
creD_time

In [ ]:
np.argsort(np.array(creDate))

In [ ]:
dd = pd.DataFrame({'file': np.array(photoList), 'time': np.array(creDate), 'time2':creD_time})
dd

In [ ]:
dd.ix[np.argsort(np.array(creDate)), :]

In [ ]:
sorted(photoList, reverse = False)

In [ ]:
x = np.array([3, 1, 2])
ind = np.argsort(x)

In [ ]:
x[ind]

In [ ]:
ind

In [ ]:
filelist = [ f for f in os.listdir(".") if f.endswith(".png") and not f.startwith(r".") ]